GradientBoostedTrees to predict popularity

In [10]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [11]:
df = pd.read_csv('project_preprocess.csv')
df.head()

,day_elapse,achievements,average_playtime,median_playtime,price,positive_percentage,english_0,english_1,required_age_0,required_age_3,...,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,popularity,positive_ratings,negative_ratings
0,-7.584710,-0.128306,9.557829,0.072624,0.141186,1.110537,0,1,1,0,...,0,0,0,0,0,0,0,Popular,124534,3339
1,-8.329745,-0.128306,0.069619,-0.035710,-0.265175,0.536451,0,1,1,0,...,0,0,0,0,0,0,0,Popular,3318,633
2,-6.414491,-0.128306,0.020358,-0.047606,-0.265175,0.775589,0,1,1,0,...,0,0,0,1,0,0,0,Popular,3416,398
3,-7.312387,-0.128306,0.059220,0.016120,-0.265175,0.480095,0,1,1,0,...,0,0,0,0,0,0,0,Popular,1273,267
4,-8.054853,-0.128306,0.259548,0.114258,-0.265175,0.999691,0,1,1,0,...,0,0,0,0,0,0,0,Popular,5250,288


In [12]:
(df_train,df_test) = train_test_split(df,train_size=0.8,test_size=0.2,random_state=0)

In [13]:
X_train = df_train.drop('popularity', axis = 1).drop('positive_ratings', axis = 1)
y_train = df_train['popularity']
X_test = df_test.drop('popularity', axis = 1).drop('positive_ratings', axis = 1)
y_test = df_test['popularity']

In [14]:
X_train.head()

,day_elapse,achievements,average_playtime,median_playtime,price,positive_percentage,english_0,english_1,required_age_0,required_age_3,...,tag_Web Publishing,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,negative_ratings
9267,0.015931,0.058841,-0.081995,-0.062050,0.623739,-0.753527,0,1,1,0,...,0,0,0,0,0,0,0,0,0,6
19313,0.492496,-0.099950,-0.081995,-0.062050,-0.557247,0.152070,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2
26925,1.060264,-0.128306,-0.081995,-0.062050,-0.771856,-0.204680,0,1,1,0,...,0,0,0,0,0,0,0,0,0,7
121,-4.734309,-0.046075,6.717113,0.734522,0.115788,1.037266,0,1,1,0,...,0,0,0,0,0,0,0,0,0,16433
2092,-0.992435,-0.077266,-0.081995,-0.062050,3.036506,-0.374186,0,1,1,0,...,0,0,0,0,0,0,0,0,0,135


In [15]:
y_train.head()

9267     Low popularity
19313    Low popularity
26925    Low popularity
121             Popular
2092            Average
Name: popularity, dtype: object

In [7]:
tr = GradientBoostingClassifier()
tr.fit(X_train, y_train)

GradientBoostingClassifier()

In [8]:
tr.score(X_test, y_test).round(3)

0.832

Baseline was: 0.687

In [16]:
tr = GradientBoostingClassifier()
grid = {'n_estimators':[10,100,200], 'max_depth' : [1,5,10], 'learning_rate':[.1,.05]}
grid_search = GridSearchCV(tr, param_grid = grid, return_train_score = True, n_jobs=-1)

grid_search.fit(X_train,y_train)

ValueError: Length of values (18) does not match length of index (3)

In [22]:
print('         best B =',grid_search.best_params_)
print('  validation R2 =',grid_search.best_score_.round(3))

         best B = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
  validation R2 = 0.887


In [18]:
test_score = grid_search.best_estimator_.score(X_test, y_test).round(3)
test_score

0.891

Baseline was: 0.687

In [23]:
from sklearn.metrics import confusion_matrix

In [24]:
yh = grid_search.predict(X_test)

In [25]:
y = y_test.astype('category')
cols = y.cat.categories
M = confusion_matrix(y, yh)
ConfusionMatrix = pd.DataFrame(M, index=cols, columns=cols)
ConfusionMatrix['total'] = ConfusionMatrix.sum(axis=1)
ConfusionMatrix

,Average,Low popularity,Popular,total
Average,619,227,90,936
Low popularity,152,3591,3,3746
Popular,113,7,613,733


This estimator is able to accurately predict Average!